In [ ]:
from imageai.Detection import ObjectDetection
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img, save_img
import os
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
def get_detector(model='yolo'):
    execution_path = os.getcwd()
    detector = ObjectDetection()
    detector.setModelTypeAsYOLOv3()
    detector.setModelPath(os.path.join(execution_path , './modules/' + model + '.h5'))
    detector.loadModel()
    custom_objects = detector.CustomObjects(car=True)
    return detector, custom_objects, execution_path
    
def filter_files(files_list, filter_by = None):
    try:
        files_list.remove('.DS_Store')
    except:
        x=1
        
    if filter_by is not None:
        mask = np.logical_not(np.isin(files_list, filter_by))
        result = []
        for f, m in zip(files_list, mask):
            if m == True:
                result.append(f)

        return result
    else:
        return files_list

def get_box(detections, size):
    img_area = size[0] * size[1]
    
    area = 0
    points = None
    for detection in detections:
        current_area = (detection['box_points'][2] - detection['box_points'][0]) * (detection['box_points'][3] - detection['box_points'][1])
        if current_area > area:
            area = current_area
            points = detection['box_points']
            
    if area / img_area < 0.2:
        return None
    else:
        return points

def crop_dir(detector, custom_objects, execution_path, source_path, target_path, directory):
    if os.path.exists(target_path + directory) == False:
        os.mkdir(target_path + directory)
    
    filenames = os.listdir(source_path + directory)
    filenames = filter_files(filenames)
    
    for filename in filenames:
        detections = detector.detectCustomObjectsFromImage(
            custom_objects=custom_objects,
            input_image=os.path.join(execution_path , source_path + directory + '/' + filename), 
            minimum_percentage_probability=10,
            extract_detected_objects=False
        )
        img = load_img(source_path + directory + '/' + filename)
        box = get_box(detections, img.size)
        cropped_img = img.crop(box)
        save_img(target_path + directory + '/' + filename, cropped_img)
    
def crop(source_path, target_path, dirs, model='yolo'):
    target_dirs = os.listdir(target_path)
    dirs = filter_files(dirs, target_dirs)

    if os.path.exists(target_path) == False:
        os.mkdir(target_path)
        
    detector, custom_objects, execution_path = get_detector(model)
    
    for i, directory in enumerate(dirs):
        print('Directory', str(i), 'of', str(len(dirs)), '...')
        crop_dir(detector, custom_objects, execution_path, source_path, target_path, directory)
        print('Images in directory', str(i), '"' + directory + '"', 'cropped')

In [ ]:
dirs = ['Bentley Continental_Flying_Spur Sedan 2007',
 'Audi TT Coupe 2012 RS',
 'Buick Rainier SUV 2007',
 'Audi A5 Coupe 2012 S5',
 'Chevrolet HHR_SS Hatchback 2010',
 'Chevrolet Corvette Coupe 2012 ZR1',
 'Audi 100 Wagon 1994',
 'Chevrolet Avalanche Crew_Cab 2012',
 'Acura ZDX SUV 2012',
 '.DS_Store',
 'Audi TT Coupe 2011',
 'Acura TSX Sedan 2012',
 'BMW X5 SUV 2007',
 'Chevrolet Express_1 Van 2007',
 'Bentley Continental_2 Coupe 2012',
 'BMW X3 SUV 2012',
 'BMW 1_Series Coupe 2012',
 'Audi A5 Convertible 2012',
 'Aston_Martin V8_Vantage Convertible 2012',
 'BMW M3 Coupe 2012',
 'Bentley Mulsanne Sedan 2011',
 'Chevrolet Corvette Convertible 2012',
 'Audi 100 Sedan 1994',
 'BMW 6_Series Convertible 2007',
 'Audi R8 Coupe 2012',
 'Chevrolet Express_1 Cargo_Van 2007',
 'Bugatti Veyron Convertible 2009',
 'Acura Integra_Type_R Coupe 2001',
 'Audi V8 Sedan 1994',
 'Buick Enclave SUV 2012',
 'Audi A5 Coupe 2012',
 'Bugatti Veyron Coupe 2009',
 'BMW Z4 Convertible 2012',
 'Audi S6 Sedan 2011',
 'BMW 1_Series Convertible 2012',
 'Chevrolet Express_2 Van 2019',
 'Cadillac SRX SUV 2012',
 'Acura TL_Type-S Sedan 2008',
 'Acura RL Sedan 2012',
 'AM Hummer SUV 2000',
 'Aston_Martin V8_Vantage Coupe 2012',
 'BMW 3_Series_2 Wagon 2010',
 'Audi TT Coupe 2012 S',
 'Aston_Martin Virage Convertible 2012',
 'Audi S4 Sedan 2012',
 'Cadillac Escalade_EXT Crew_Cab 2007',
 'Audi A5 Convertible 2012 S5',
 'Bentley Continental_1 Coupe 2007',
 'Chevrolet Cobalt_SS Coupe 2010',
 'BMW 3_Series_3 Wagon 2014',
 'Buick Verano Sedan 2012',
 'Buick Regal_GS Sedan 2012',
 'Cadillac CTS-V Sedan 2012',
 'BMW 6_Series Convertible 2010 M6',
 'Acura TL Sedan 2012',
 'Audi A4 Sedan 2007 S4',
 'Aston_Martin Virage Coupe 2012',
 'Bentley Continental_1 Convertible 2012',
 'BMW M5 Sedan 2010',
 'Chevrolet Express_2 Cargo_Van 2019',
 'Chevrolet Camaro Convertible 2012',
 'BMW 5_Series Sedan 2012',
 'BMW X6 SUV 2012',
 'Chevrolet Corvette Coupe 2007 Z06',
 'Bentley Arnage Sedan 2009',
 'Audi A4 Convertible 2008 RS_4',
 'BMW 3_Series_3 Sedan 2012']

In [ ]:
crop(
    './stanford-car-dataset-by-classes-folder/car_data/', 
    './stanford-car-dataset-by-classes-folder/cropped_car_data/',
    dirs,
    'yolo',
)

## Preview

In [ ]:
execution_path = os.getcwd()

detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(os.path.join(execution_path , "./modules/yolo.h5"))
detector.loadModel()
custom_objects = detector.CustomObjects(car=True)

detections = detector.detectCustomObjectsFromImage(
    custom_objects=custom_objects,
    input_image=os.path.join(execution_path , "./stanford-car-dataset-by-classes-folder/car_data/AM Hummer SUV 2000/00163.jpg"), 
    minimum_percentage_probability=30,
    output_image_path=os.path.join(execution_path , "image2new.jpg"),
    #extract_detected_objects=True
)

print(detections)
img = load_img('./stanford-car-dataset-by-classes-folder/car_data/AM Hummer SUV 2000/00163.jpg')
print(img.size)
w,h = img.size

box = get_box(detections, img.size)
print(box)
crop_img = img.crop(box)
plt.imshow(crop_img)